In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
def getEveryDay(begin_date,end_date):
    date_list = []
    date_flag = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        if begin_date.strftime("%m-%d") >= '11-01' and begin_date.strftime("%m-%d") <= '11-11' or \
                begin_date.strftime("%m-%d") >= '12-01' and begin_date.strftime("%m-%d") <= '12-12' or \
                   begin_date.strftime("%m-%d") >= '10-01' and begin_date.strftime("%m-%d") <= '10-07' :
            date_flag.append(2)
        elif begin_date.strftime("%w") == '0' or begin_date.strftime("%w") == '6':
            date_flag.append(1)
        else:
            date_flag.append(0)
        begin_date += datetime.timedelta(days=1)
    return date_list, date_flag

In [15]:
data_dir = '../data/'

begin_date, end_date = '2015-01-01', '2019-12-17'
begin_year, end_year = 2015, 2019

categories_purchase = []
categories_cart = []
area_purchase = []
area_cart = []

dates, date_flag = getEveryDay(begin_date, end_date)
areas = ['怀柔区', '大兴区', '昌平区', '通州区', '石景山区', '延庆区', '顺义区', '西城区', '平谷区', '宣武区', '东城区', '丰台区', '朝阳区', '门头沟', '崇文区', '房山区', '密云区', '海淀区']   
categories = ['厨具', '服饰内衣', '家用电器', '鞋靴', '家装建材', '宠物生活', '电脑、办公', '图书', '家具', '手机通讯', '玩具乐器', '本地生活/旅游出行', '钟表', '酒类', '影视', '医药保健', '农资园艺', '美妆护肤', '整车', '文娱', '礼品', '珠宝首饰', '食品饮料', '家居日用', '汽车用品', '运动户外', '生鲜', '音乐', '母婴', '数码']

In [14]:
print(set(categories1)-set(categories))

{'家具家装'}


In [9]:
for year in range(begin_year, end_year+1):
    purchase_data = pd.read_csv(data_dir + 'purchase-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    # select categoties of products in purchase data
    categories = purchase_data.groupby('一级类目').nunique()
    categories_purchase.append(set(categories['一级类目'].index.tolist()))
    # select all area in purchase data
    areas = purchase_data.groupby('市').nunique()
    area_purchase.append(set(areas['市'].index.tolist()))

    cart_data = pd.read_csv(data_dir + 'shoppingcart-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    # select categoties of products in shopping cart data
    categories = cart_data.groupby('一级类目').nunique()
    categories_cart.append(set(categories['一级类目'].index.tolist()))
    # select all area in shopping cart data
    areas = cart_data.groupby('市').nunique()
    area_cart.append(set(areas['市'].index.tolist()))

categories_purchase = categories_purchase[0] & categories_purchase[1] & categories_purchase[2] & categories_purchase[3] & categories_purchase[4]
categories_purchase = categories_purchase - set(['\\N', '测试分类'])
area_purchase = area_purchase[0] & area_purchase[1] & area_purchase[2] & area_purchase[3] & area_purchase[4]

categories_cart = categories_cart[0] & categories_cart[1] & categories_cart[2] & categories_cart[3] & categories_cart[4]
categories_cart = categories_cart - set(['\\N', '测试分类'])
area_cart = area_cart[0] & area_cart[1] & area_cart[2] & area_cart[3] & area_cart[4]

categories = list(categories_purchase & categories_cart)
areas = list(area_purchase & area_cart)

print(categories, len(categories))
print(areas, len(areas))


['服饰内衣', '汽车用品', '厨具', '玩具乐器', '数码', '整车', '音乐', '母婴', '钟表', '农资园艺', '文娱', '生鲜', '鞋靴', '家装建材', '运动户外', '酒类', '本地生活/旅游出行', '礼品', '电脑、办公', '食品饮料', '宠物生活', '珠宝首饰', '影视', '医药保健', '家用电器', '手机通讯', '美妆护肤', '图书', '家具家装', '家具', '家居日用'] 31
['门头沟', '丰台区', '密云区', '石景山区', '崇文区', '延庆区', '东城区', '西城区', '大兴区', '宣武区', '朝阳区', '通州区', '平谷区', '怀柔区', '顺义区', '昌平区', '海淀区', '房山区'] 18


In [17]:
for year in range(begin_year, end_year + 1):
    purchase_data = pd.read_csv(data_dir + 'purchase-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    purchase_data = purchase_data.drop(['省', '区'], axis=1)
    purchase_data['一级类目'] = purchase_data['一级类目'].str.strip()
    purchase_data = purchase_data.groupby(['日期', '市', '一级类目']).sum().reset_index()
    
    cart_data = pd.read_csv(data_dir + 'shoppingcart-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    cart_data = cart_data.drop(['省', '区'], axis=1)
    cart_data['一级类目'] = cart_data['一级类目'].str.strip()
    cart_data = cart_data.groupby(['日期', '市', '一级类目']).sum().reset_index() 
    print(len(purchase_data), len(cart_data))
    
    if year == begin_year:
        purchase = purchase_data
        cart = cart_data
    else:
        purchase = pd.concat([purchase, purchase_data])
        cart = pd.concat([cart, cart_data])
print(len(purchase), len(cart))

228973 108709
232995 186203
250604 221889
263918 265114
309433 260485
1285923 1042400


In [25]:
def spatial_partition(purchase_data, cart_data, cate_list, area_flag, date_list):
    for area in area_flag:
        area_purchase = []
        for date in date_list:
            date_purchase = []
            for cate in cate_list:
                entry = purchase_data[(purchase_data['日期'] == date) & (purchase_data['市'] == area) & (purchase_data['一级类目'] == cate)]['数量']
                if len(entry) == 0:
                    date_purchase.append(0)
                else:
                    date_purchase.append(int(entry))
            area_purchase.append(date_purchase)
            print(area, date)
        np.save('../data/spatial/purchase/' + area + '.npy', np.array(area_purchase))

In [ ]:
spatial_partition(purchase, cart, categories, areas, dates)

怀柔区 2015-01-01
怀柔区 2015-01-02
怀柔区 2015-01-03
怀柔区 2015-01-04
怀柔区 2015-01-05
怀柔区 2015-01-06
怀柔区 2015-01-07
怀柔区 2015-01-08
怀柔区 2015-01-09
怀柔区 2015-01-10
怀柔区 2015-01-11
怀柔区 2015-01-12
怀柔区 2015-01-13
怀柔区 2015-01-14
怀柔区 2015-01-15
怀柔区 2015-01-16
怀柔区 2015-01-17
怀柔区 2015-01-18
怀柔区 2015-01-19
怀柔区 2015-01-20
怀柔区 2015-01-21
怀柔区 2015-01-22
怀柔区 2015-01-23
怀柔区 2015-01-24
怀柔区 2015-01-25
怀柔区 2015-01-26
怀柔区 2015-01-27
怀柔区 2015-01-28
怀柔区 2015-01-29
怀柔区 2015-01-30
怀柔区 2015-01-31
怀柔区 2015-02-01
怀柔区 2015-02-02
怀柔区 2015-02-03
怀柔区 2015-02-04
怀柔区 2015-02-05
怀柔区 2015-02-06
怀柔区 2015-02-07
怀柔区 2015-02-08
怀柔区 2015-02-09
怀柔区 2015-02-10
怀柔区 2015-02-11
怀柔区 2015-02-12
怀柔区 2015-02-13
怀柔区 2015-02-14
怀柔区 2015-02-15
怀柔区 2015-02-16
怀柔区 2015-02-17
怀柔区 2015-02-18
怀柔区 2015-02-19
怀柔区 2015-02-20
怀柔区 2015-02-21
怀柔区 2015-02-22
怀柔区 2015-02-23
怀柔区 2015-02-24
怀柔区 2015-02-25
怀柔区 2015-02-26
怀柔区 2015-02-27
怀柔区 2015-02-28
怀柔区 2015-03-01
怀柔区 2015-03-02
怀柔区 2015-03-03
怀柔区 2015-03-04
